In [ ]:
using Turing, AdvancedVI, Distributions, Random, Plots, LinearAlgebra, DelimitedFiles
using ReverseDiff, Memoization
using StatsFuns

Random.seed!(1234);
Turing.setadbackend(:reversediff);
Turing.setrdcache(true);

In [1]:
sunspot = readdlm("data/SN_y_tot_V2.0.csv", ';')
select_1945_2020 = filter((s) -> s[1] > 1945 && s[1] < 2021, collect(eachrow(sunspot)))
dataset = map((s) -> round(s[2]), select_1945_2020);
not_rounded = map((s) -> s[2], select_1945_2020);


UndefVarError: UndefVarError: readdlm not defined

In [ ]:
@model model(y) = begin
    n = length(y)
    b ~ Gamma(1000, 2)
    z = Vector{Float64}(undef, N)
    z[1] ~ Gamma(1, 1/b) # Shape Scale parameterization in Distributions.jl
    y[1] ~ Poisson(z[1])
    for i = 2:n
        z[i] ~ Gamma(softplus(z[i-1]), 1/b)
        y[i] ~ Poisson(z[i])
    end
end

In [ ]:
# Instantiate model
sc_model = model(dataset);

In [ ]:
runtime = @elapsed chain = sample(sc_model, NUTS(200, 0.65), 1000);

In [ ]:
ids = findall(map(name -> occursin("z", string(name)), names(chain)));

In [ ]:
mean_list = []
var_list = []
for n=2:N+1
    push!(mean_list, mean(chain[:,n,:]))
    push!(var_list, var(chain[:,n,:]))
end

In [ ]:
scatter(collect(1:N),dataset,color=:black,size=(900,200),legend=:topright, 
        xtick=([1, 12, 23, 34, 45, 56, 67, 76], [1945, 1956, 1967, 1978, 1989, 2000, 2011, 2020]), 
        xlabel="Years", ylabel="Sunspots", label="Observations", left_margin = 10Plots.mm, bottom_margin = 5Plots.mm)
plot!(mean_list,ribbon=var_list,label="Estimations", lw=2, size=(1000,400),color=:green,fillalpha=0.2)

In [ ]:
open("results/nuts_sunspot_results.csv", "w") do io
    writedlm(io, [mean_list var_list])
end

In [ ]:
sqrt(mean((not_rounded - mean_list).^2))